In [ ]:
word2vec_path = "/ilab/users/kc1026/Documents/cs543/GoogleNews-vectors-negative300.bin.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [874]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping
import gensim
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import codecs
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

stop_words = set(stopwords.words('english'))
EMBEDDING_DIM = 300 # how big is each word vector
MAX_VOCAB_SIZE = 175303 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 150 # max number of words in a comment to use

#training params
batch_size = 256 
num_epochs = 20 

In [875]:
df = pd.read_csv("/ilab/users/kc1026/Documents/cs543/emoji.csv", sep=',', header=0)
train, test = train_test_split(df, test_size=0.1)

In [876]:
tokenizer = RegexpTokenizer(r'\w+')
train['text'] = train['text'].apply(lambda x: re.sub("[^a-zA-Z']", ' ', x))
test['text'] = test['text'].apply(lambda x: re.sub("[^a-zA-Z']", ' ', x))

/usr/lib/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/lib/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [878]:
train['text'][2]

'When I was driving home after  several days of hard work  there was a motorist ahead of me who was driving at    km hour and refused  despite his low speeed to let me overtake '

In [833]:
# all_training_words = [word for token in train["token"] for word in token]
# training_sentence_lengths = [len(token) for token in train["token"]]
# TRAINING_VOCAB = sorted(list(set(all_training_words)))
# print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
# print("Max sentence length is %s" % max(training_sentence_lengths))

In [870]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(train["text"].tolist())
x_training_sequences = tokenizer.texts_to_sequences(train["text"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

# set max length of sequences, now all data has the same length of 300
train_cnn_data = pad_sequences(x_training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

num_words = len(train_word_index) + 1
train_embedding_weights = np.zeros((num_words, EMBEDDING_DIM))
for word,index in train_word_index.items():
    if word in word2vec:
        train_embedding_weights[index,:] = word2vec[word]     


Found 8543 unique tokens.


array([[   0,    0,    0, ...,   11,   15,  170],
       [   0,    0,    0, ...,    1,   11,  576],
       [   0,    0,    0, ..., 4368,  617,  385],
       ...,
       [   0,    0,    0, ...,    4,   81,   59],
       [   0,    0,    0, ..., 8543,   12,  365],
       [   0,    0,    0, ..., 2355,    5,   34]], dtype=int32)

In [835]:
test_sequences = tokenizer.texts_to_sequences(test["text"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = test_cnn_data

In [851]:
x_train = train_cnn_data
y_train = np_utils.to_categorical(train['label'].values)

In [860]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index=7, trainable=False, extra_conv=True):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.25)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.25)(pool)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.25)(x)
    # Finally, we feed the output into a Sigmoid layer.
    # The reason why sigmoid is used is because we are trying to achieve a binary classification(1,0) 
    # for each of the 6 labels, and the sigmoid function will squash the output between the bounds of 0 and 1.
    preds = Dense(labels_index, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.summary()
    return model

In [853]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, num_words, EMBEDDING_DIM)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_52 (Embedding)        (None, 150, 300)     2563200     input_44[0][0]                   
__________________________________________________________________________________________________
conv1d_172 (Conv1D)             (None, 148, 128)     115328      embedding_52[0][0]               
__________________________________________________________________________________________________
conv1d_173 (Conv1D)             (None, 147, 128)     153728      embedding_52[0][0]               
__________________________________________________________________________________________________
conv1d_174

In [854]:
#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [855]:
hist = model.fit(x_train, y_train, epochs=num_epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, batch_size=batch_size)

Train on 6058 samples, validate on 674 samples
Epoch 1/20
6058/6058 [==============================] - 14s 2ms/step - loss: 2.1983 - acc: 0.1651 - val_loss: 1.9304 - val_acc: 0.1929
Epoch 2/20
6058/6058 [==============================] - 7s 1ms/step - loss: 1.9465 - acc: 0.1722 - val_loss: 1.9363 - val_acc: 0.1706
Epoch 3/20
6058/6058 [==============================] - 7s 1ms/step - loss: 1.9254 - acc: 0.1892 - val_loss: 1.9073 - val_acc: 0.2033
Epoch 4/20
6058/6058 [==============================] - 7s 1ms/step - loss: 1.9148 - acc: 0.1961 - val_loss: 1.8836 - val_acc: 0.2270
Epoch 5/20
6058/6058 [==============================] - 8s 1ms/step - loss: 1.8906 - acc: 0.2202 - val_loss: 1.8413 - val_acc: 0.2804
Epoch 6/20
6058/6058 [==============================] - 7s 1ms/step - loss: 1.8792 - acc: 0.2230 - val_loss: 1.8183 - val_acc: 0.2596
Epoch 7/20
6058/6058 [==============================] - 8s 1ms/step - loss: 1.8528 - acc: 0.2364 - val_loss: 1.7410 - val_acc: 0.2878
Epoch 8/20
605

In [856]:
y_predict = model.predict(x_test, batch_size=64, verbose=1)
y_actual = test['label'].tolist()

748/748 [==============================] - 3s 5ms/step


In [857]:
y_predict_top1 = []
for i in range(0, len(y_predict)):
    max_index = 0
    max_value = 0.0
    for j in range(0, len(y_predict[0])):
        if y_predict[i][j] > max_value:
            max_value = y_predict[i][j]
            max_index = j

    y_predict_top1.append(max_index)

In [858]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_actual, y_predict_top1)

In [859]:
accuracy

0.2660427807486631

In [887]:
import nltk
nltk.corpus.brown.tagged_words(tagset='CONJ')

[('The', 'UNK'), ('Fulton', 'UNK'), ('County', 'UNK'), ...]